In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import sys
sys.path.append("../")
# !conda install --yes --prefix {sys.prefix} gensim


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [7]:
# Setting up N2VEC parameters 
from topsy_turvy.comparison_methods.n2vec_embedding import compute_embedding

input_folder  = "../datasets/Analysis/"
output_folder = "../datasets/Analysis/"
network       = "fly_ppi"

In [14]:
import pandas as pd
import json
# Given a network file 
"""
net_file       = f"{input_folder}/{network}.tsv"
json_file      = f"{output_folder}/{network}_nodemap.json"
df    = pd.read_csv(net_file, sep = "\t", header = None)
df.columns = ["p", "q", "weight"]
nodes     = set(df["p"]).union(set(df["q"]))
nodes_map = {node: i for i, node in enumerate(nodes)}
with open(json_file, "w") as jf:
    json.dump(nodes_map, jf)
df_annotated = df.replace({"p": nodes_map, "q": nodes_map})
df_annotated.to_csv(f"{output_folder}/annotate_{network}.tsv", 
                    header = False, 
                    index = False, 
                    sep = "\t")
df_annotated
"""

,p,q,weight
0,163,2349,0.898801
1,3283,1459,0.004207
2,1725,1565,0.394838
3,455,2894,0.004177
4,245,19,0.004201
...,...,...,...
27465,351,6,0.004178
27466,3420,1722,0.987240
27467,1559,1386,0.985707
27468,2287,1202,0.004253


In [38]:
# NODE2VEC ARGS
args      = {}
args["p"] = 1
args["q"] = 1
args["dimensions"] = 300
args["num_walks"]  = 10
args["intermediate_file_loc"] = f"{output_folder}/{network}_text_dim_{args['dimensions']}.emb"
args["final_emb"]             = f"{output_folder}/{network}_emb_dim_{args['dimensions']}.npy"

In [16]:
df_annotated = pd.read_csv(f"{output_folder}/annotate_{network}.tsv", sep = "\t", header = None)
df_annotated

,0,1,2
0,163,2349,0.898801
1,3283,1459,0.004207
2,1725,1565,0.394838
3,455,2894,0.004177
4,245,19,0.004201
...,...,...,...
27465,351,6,0.004178
27466,3420,1722,0.987240
27467,1559,1386,0.985707
27468,2287,1202,0.004253


In [17]:
edgelist = list(df_annotated.to_records(index = False))

In [28]:
embeddings = compute_embedding(edgelist, args)

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
{'p': 1, 'q': 1, 'dimensions': 300, 'num_walks': 10, 'intermediate_file_loc': '../datasets/Analysis//fly_ppi_text_dim_300.emb', 'input': 'karate.edgelist', 'walk-length': 80, 'num-walks': 10, 'window-size': 10, 'iter': 1, 'workers': 8, 'weighted': True, 'unweighted': False, 'directed': False, 'undirected': True}
Here


../topsy_turvy/comparison_methods/n2vec_embedding.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  emb = np.array(emb)


In [34]:
import numpy as np
n_nodes = 0
n_emb   = 0
with open(args["intermediate_file_loc"], "r") as ef:
    n_nodes, n_emb = ef.readline().strip().split(" ")
    n_nodes        = int(n_nodes)
    n_emb          = int(n_emb)
    embeddings     = np.zeros((n_nodes, n_emb))
    
    for line in ef:
        emb        = line.strip().split(" ")
        index      = int(emb[0])
        features   = [float(f_i) for f_i in emb[1:]]
        embeddings[index] = features

In [39]:
np.save(args["final_emb"], embeddings) 

In [40]:
emb = np.load(args["final_emb"])